In [8]:
import earthaccess

username = "ayush.pix"
pw = "Tumunichdon#pix24"

earthaccess.login()

In [11]:
import os
import subprocess
import xarray as xr
import pandas as pd
from glob import glob

# ===============================
# User Input Section
# ===============================
print("Enter latitude bounds (min_lat,max_lat):")
lat_min, lat_max = map(float, input().split(','))

print("Enter longitude bounds (min_lon,max_lon):")
lon_min, lon_max = map(float, input().split(','))

print("Enter time period (start_date,end_date) in YYYY-MM-DD:")
start_date, end_date = input().split(',')

output_folder = 'terra_downloads'
csv_folder = 'terra_csv_output'
os.makedirs(output_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)

# ===============================
# 1️⃣ Bash download script as Python subprocess
# ===============================
# Products dictionary: product -> example base URL (LP DAAC / Earthdata)
products = {
    'ASTER_AST_08': 'https://e4ftl01.cr.usgs.gov/ASTT/AST_08.003',
    'MODIS_MOD11A1': 'https://e4ftl01.cr.usgs.gov/MOLT/MOD11A1.061',
    'CERES_EBAF': 'https://ceres.larc.nasa.gov/data/EBAF',
    'MOPITT_CO': 'https://terra.nasa.gov/data/mopitt-data',
    'MISR_L2': 'https://e4ftl01.cr.usgs.gov/MISR/MIL2ASAE.003'
}

print("\n[INFO] Downloading files via wget (you must have Earthdata credentials in .netrc)...")

for product, url in products.items():
    # Here we demonstrate downloading the latest file
    # In practice, loop over date range, construct DOY/file paths per product
    example_file = f"{product}_example.hdf"
    wget_cmd = f"""wget --auth-no-challenge=on -O {output_folder}/{example_file} {url}"""
    print(f"Downloading {product}...")
    subprocess.run(wget_cmd, shell=True)

# ===============================
# 2️⃣ Convert downloaded files to CSV
# ===============================
"""
variable_map = {
    'MOD11A1': 'LST_Day_1km',
    'ASTER_AST_08': 'Surface_Temperature',
    'CERES_EBAF': 'toa_sw_allsky',
    'MOPITT_CO': 'CO',
    'MISR_L2': 'Radiance'
}
"""
variable_map = {
    'MOD11A1': 'LST_Day_1km'}
    
data_files = glob(os.path.join(output_folder, '*.[hn]df'))
csv_files = []

def convert_to_csv(file_path, variable_name, output_dir):
    ds = xr.open_dataset(file_path)
    if {'lat','lon'}.issubset(ds.coords):
        ds_subset = ds.sel(lat=slice(lat_min, lat_max),
                           lon=slice(lon_min, lon_max))
    else:
        ds_subset = ds
    df = ds_subset[variable_name].to_dataframe().reset_index()
    base_name = os.path.basename(file_path).split('.')[0]
    csv_file = os.path.join(output_dir, f"{base_name}_{variable_name}.csv")
    df.to_csv(csv_file, index=False)
    return csv_file

for file_path in data_files:
    for key in variable_map:
        if key in os.path.basename(file_path):
            try:
                csv_file = convert_to_csv(file_path, variable_map[key], csv_folder)
                csv_files.append(csv_file)
                print(f"Converted {file_path} → {csv_file}")
            except Exception as e:
                print(f"Skipping {file_path}: {e}")

# ===============================
# 3️⃣ Load CSVs into DataFrames
# ===============================
dataframes = {}
for csv_file in csv_files:
    df_name = os.path.basename(csv_file).replace('.csv','')
    df = pd.read_csv(csv_file)
    dataframes[df_name] = df
    print(f"Loaded CSV {csv_file} into DataFrame '{df_name}'")

print("\n[INFO] All CSVs loaded into DataFrames. Access via the 'dataframes' dictionary.")


Enter latitude bounds (min_lat,max_lat):
Enter longitude bounds (min_lon,max_lon):
Enter time period (start_date,end_date) in YYYY-MM-DD:

[INFO] Downloading files via wget (you must have Earthdata credentials in .netrc)...

[INFO] All CSVs loaded into DataFrames. Access via the 'dataframes' dictionary.


In [ ]:
import os
import requests
import xarray as xr
import pandas as pd
from glob import glob

# ===============================
# 1️⃣ User Inputs
# ===============================
lat_min, lat_max = map(float, input("Enter latitude bounds (min_lat,max_lat): ").split(','))
lon_min, lon_max = map(float, input("Enter longitude bounds (min_lon,max_lon): ").split(','))

start_date, end_date = input("Enter time period (start_date,end_date) in YYYY-MM-DD: ").split(',')

username = input("Enter your NASA Earthdata username: ")
password = input("Enter your NASA Earthdata password: ")

output_folder = 'terra_downloads'
csv_folder = 'terra_csv_output'
os.makedirs(output_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)

# ===============================
# 2️⃣ Define Product URLs
# ===============================
# NOTE: You must replace these example URLs with actual HDF/NetCDF file URLs
# from LP DAAC / CERES / MOPITT data pool for your date range
"""
products = {
    'ASTER_AST_08': ['https://e4ftl01.cr.usgs.gov/ASTT/AST_08.003/ASTER_sample_file.hdf'],
    'MODIS_MOD11A1': ['https://e4ftl01.cr.usgs.gov/MOLT/MOD11A1.061/MOD11A1_sample_file.hdf'],
    'CERES_EBAF': ['https://ceres.larc.nasa.gov/data/EBAF/sample_file.nc'],
    'MOPITT_CO': ['https://terra.nasa.gov/data/mopitt-data/sample_file.nc']
}
"""
products = {
    'ASTER_AST_08': ['https://www.earthdata.nasa.gov/data/catalog/lpdaac-ecs-ast-08-003'],
    'MODIS_MOD11A1': ['https://e4ftl01.cr.usgs.gov/MOLT/MOD11A1.061/MOD11A1_sample_file.hdf'],
    'CERES_EBAF': ['https://ceres.larc.nasa.gov/data/EBAF/sample_file.nc'],
    'MOPITT_CO': ['https://terra.nasa.gov/data/mopitt-data/sample_file.nc']
}

# ===============================
# 3️⃣ Download Files with Requests
# ===============================
print("\n[INFO] Downloading files from NASA Earthdata...")
for product, urls in products.items():
    for url in urls:
        local_filename = os.path.join(output_folder, os.path.basename(url))
        if not os.path.exists(local_filename):
            print(f"Downloading {url}...")
            try:
                response = requests.get(url, auth=(username, password), stream=True)
                response.raise_for_status()
                with open(local_filename, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"Saved: {local_filename}")
            except Exception as e:
                print(f"Failed to download {url}: {e}")
        else:
            print(f"File already exists: {local_filename}")

# ===============================
# 4️⃣ Convert HDF/NetCDF to CSV
# ===============================
# Map product names to variable names
variable_map = {
    'MOD11A1': 'LST_Day_1km',
    'ASTER_AST_08': 'Surface_Temperature',
    'CERES_EBAF': 'toa_sw_allsky',
    'MOPITT_CO': 'CO'
}

def convert_to_csv(file_path, variable_name, output_dir):
    try:
        ds = xr.open_dataset(file_path)
    except Exception as e:
        print(f"Cannot open {file_path}: {e}")
        return None

    # Subset by lat/lon if available
    if {'lat','lon'}.issubset(ds.coords):
        ds_subset = ds.sel(lat=slice(lat_min, lat_max),
                           lon=slice(lon_min, lon_max))
    else:
        ds_subset = ds

    # Convert to DataFrame
    try:
        df = ds_subset[variable_name].to_dataframe().reset_index()
    except Exception as e:
        print(f"Cannot extract variable {variable_name} from {file_path}: {e}")
        return None

    base_name = os.path.basename(file_path).split('.')[0]
    csv_file = os.path.join(output_dir, f"{base_name}_{variable_name}.csv")
    df.to_csv(csv_file, index=False)
    return csv_file

# Process all downloaded files
data_files = glob(os.path.join(output_folder, '*.[hn]df'))
csv_files = []
for file_path in data_files:
    for key in variable_map:
        if key in os.path.basename(file_path):
            csv_file = convert_to_csv(file_path, variable_map[key], csv_folder)
            if csv_file:
                csv_files.append(csv_file)
                print(f"Converted {file_path} → {csv_file}")

# ===============================
# 5️⃣ Load CSVs into Pandas DataFrames
# ===============================
dataframes = {}
for csv_file in csv_files:
    df_name = os.path.basename(csv_file).replace('.csv','')
    df = pd.read_csv(csv_file)
    dataframes[df_name] = df
    print(f"Loaded CSV {csv_file} into DataFrame '{df_name}'")

print("\n[INFO] All CSVs loaded. Access them via the 'dataframes' dictionary.")



[INFO] Downloading files from NASA Earthdata...
Failed to download https://e4ftl01.cr.usgs.gov/ASTT/AST_08.003/ASTER_sample_file.hdf: 404 Client Error: Not Found for url: https://e4ftl01.cr.usgs.gov/ASTT/AST_08.003/ASTER_sample_file.hdf
Failed to download https://e4ftl01.cr.usgs.gov/MOLT/MOD11A1.061/MOD11A1_sample_file.hdf: 404 Client Error: Not Found for url: https://e4ftl01.cr.usgs.gov/MOLT/MOD11A1.061/MOD11A1_sample_file.hdf
Failed to download https://ceres.larc.nasa.gov/data/EBAF/sample_file.nc: 404 Client Error: Not Found for url: https://ceres.larc.nasa.gov/data/EBAF/sample_file.nc
Failed to download https://terra.nasa.gov/data/mopitt-data/sample_file.nc: 404 Client Error: Not Found for url: https://terra.nasa.gov/data/mopitt-data/sample_file.nc

[INFO] All CSVs loaded. Access them via the 'dataframes' dictionary.


In [13]:
pip install streamlit pandas


   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ----------------- ---------------------- 4.5/10.1 MB 23.4 MB/s eta 0:00:01
   ------------------------------------ --- 9.2/10.1 MB 23.1 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 21.2 MB/s  0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   ---------------------------------------- 731.2/731.2 kB 14.7 MB/s  0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------- ----------- 5.0/7.0 MB 23.6 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 20.8 MB/s  0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ------------------------ --------------- 4.2/6.9 MB 20.2 MB/s eta 0:00:01
   ---------------------------------------- 6.9/6.9 MB 18.5 MB/s  0:00:00
   ---------------------------------------- 0.0/26.1 MB ? eta -:--:--
   ------ -----------------------------

In [14]:
import streamlit as st
import pandas as pd

# --------------------------------------------------
# 1️⃣ Page configuration
# --------------------------------------------------
st.set_page_config(page_title="Rice Growth Demo", layout="wide")

# --------------------------------------------------
# 2️⃣ Session state to store inputs
# --------------------------------------------------
if "page" not in st.session_state:
    st.session_state.page = 1

def next_page():
    st.session_state.page += 1

def prev_page():
    st.session_state.page -= 1

# --------------------------------------------------
# 3️⃣ Page 1: Select Location
# --------------------------------------------------
if st.session_state.page == 1:
    st.title("🌍 Select Location")
    st.write("Please select your region and specify latitude/longitude bounds.")

    region = st.text_input("Region / City Name", placeholder="e.g., Hanoi, Vietnam")
    lat_min = st.number_input("Minimum Latitude", value=10.0)
    lat_max = st.number_input("Maximum Latitude", value=20.0)
    lon_min = st.number_input("Minimum Longitude", value=100.0)
    lon_max = st.number_input("Maximum Longitude", value=110.0)

    st.write("---")
    st.button("Next ➡️", on_click=next_page)

# --------------------------------------------------
# 4️⃣ Page 2: Select Crop
# --------------------------------------------------
elif st.session_state.page == 2:
    st.title("🌾 Select Crop")
    st.write("Choose the crop you want to analyze.")
    crop = st.selectbox("Select Crop", ["Rice"], index=0)
    st.write(f"✅ You selected: **{crop}**")

    st.write("---")
    col1, col2 = st.columns(2)
    col1.button("⬅️ Back", on_click=prev_page)
    col2.button("Next ➡️", on_click=next_page)

# --------------------------------------------------
# 5️⃣ Page 3: Compare Ideal vs Local Conditions
# --------------------------------------------------
elif st.session_state.page == 3:
    st.title("🌦️ Compare Growing Conditions for Rice")

    st.write("Below is a comparison between **ideal conditions for rice** and your region's **average annual conditions**.")

    # Ideal rice growing conditions
    ideal = {
        "Parameter": [
            "Temperature (°C)",
            "Annual Rainfall (mm)",
            "Soil Type",
            "Soil pH",
            "Sunlight (hours/day)",
            "Humidity (%)",
            "Altitude (m)"
        ],
        "Ideal for Rice": [
            "20–35",
            "1000–2000",
            "Alluvial / Clay loam",
            "5.5–7.0",
            "4–6",
            "70–80",
            "0–1500"
        ]
    }

    # Dummy “local” example data (in a real app, load from dataset or API)
    local = {
        "Parameter": [
            "Temperature (°C)",
            "Annual Rainfall (mm)",
            "Soil Type",
            "Soil pH",
            "Sunlight (hours/day)",
            "Humidity (%)",
            "Altitude (m)"
        ],
        "Your Location (Average)": [
            "29",
            "1450",
            "Clay loam",
            "6.2",
            "5.2",
            "75",
            "210"
        ]
    }

    df_ideal = pd.DataFrame(ideal)
    df_local = pd.DataFrame(local)

    col1, col2 = st.columns(2)

    with col1:
        st.subheader("✅ Ideal Rice Conditions")
        st.dataframe(df_ideal, use_container_width=True)

    with col2:
        st.subheader("📍 Your Location")
        st.dataframe(df_local, use_container_width=True)

    st.write("---")
    st.button("⬅️ Back", on_click=prev_page)


2025-10-04 17:37:29.917 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 17:37:29.919 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 17:37:29.919 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-10-04 17:37:29.920 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 17:37:29.920 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 17:37:29.922 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

In [15]:
pip install streamlit pandas requests


Note: you may need to restart the kernel to use updated packages.


In [17]:
import streamlit as st
import pandas as pd
import requests
import plotly.express as px

st.set_page_config(page_title="Crop Growth Suitability Demo", layout="wide")

# ----------------------------------------------------------------
# Crop database — easy to extend later
# ----------------------------------------------------------------
CROP_CONDITIONS = {
    "Rice": {
        "Temperature (°C)": (20, 35),
        "Annual Rainfall (mm)": (1000, 2000),
        "Soil pH": (5.5, 7.0),
        "Sunlight (MJ/m²/day)": (14, 20),
        "Relative Humidity (%)": (70, 80),
        "Altitude (m)": (0, 1500),
    },
    "Wheat": {
        "Temperature (°C)": (10, 25),
        "Annual Rainfall (mm)": (500, 1200),
        "Soil pH": (6.0, 7.5),
        "Sunlight (MJ/m²/day)": (10, 16),
        "Relative Humidity (%)": (50, 70),
        "Altitude (m)": (0, 3000),
    },
}

# ----------------------------------------------------------------
# NASA POWER API functions
# ----------------------------------------------------------------
def get_climate_data(lat, lon):
    """Fetch monthly climatology for multiple parameters from NASA POWER."""
    url = (
        f"https://power.larc.nasa.gov/api/temporal/monthly/point?"
        f"parameters=T2M,PRECTOTCORR,RH2M,ALLSKY_SFC_SW_DWN"
        f"&community=AG&longitude={lon}&latitude={lat}&format=JSON&start=2015&end=2024"
    )
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()["properties"]["parameter"]

    df = pd.DataFrame(data)
    df.index = pd.to_datetime(df.index)
    df.rename(
        columns={
            "T2M": "Temperature (°C)",
            "PRECTOTCORR": "Rainfall (mm)",
            "RH2M": "Humidity (%)",
            "ALLSKY_SFC_SW_DWN": "Sunlight (MJ/m²/day)",
        },
        inplace=True,
    )
    return df

# ----------------------------------------------------------------
# GUI Layout
# ----------------------------------------------------------------
st.title("🌾 Crop Growth Suitability Explorer")
st.write("Analyze if your location is suitable for growing different crops based on NASA POWER climate data.")

# Input controls
st.sidebar.header("User Input")
lat = st.sidebar.number_input("Latitude", value=22.0, format="%.2f")
lon = st.sidebar.number_input("Longitude", value=88.0, format="%.2f")
crop = st.sidebar.selectbox("Select Crop", list(CROP_CONDITIONS.keys()))

# ----------------------------------------------------------------
# Main content layout
# ----------------------------------------------------------------
col1, col2 = st.columns(2)

# Ideal conditions
with col1:
    st.subheader(f"✅ Ideal Conditions for {crop}")
    crop_df = pd.DataFrame(
        [(k, f"{v[0]}–{v[1]}") for k, v in CROP_CONDITIONS[crop].items()],
        columns=["Parameter", "Ideal Range"],
    )
    st.dataframe(crop_df, use_container_width=True)

# Fetch and show actual data
with col2:
    st.subheader("📍 Local Climate Data (NASA POWER)")
    try:
        df = get_climate_data(lat, lon)
        avg_data = df.mean().round(2)
        min_data = df.min().round(2)
        max_data = df.max().round(2)

        stats = pd.DataFrame(
            {"Min": min_data, "Avg": avg_data, "Max": max_data}
        ).reset_index()
        stats.rename(columns={"index": "Parameter"}, inplace=True)
        st.dataframe(stats, use_container_width=True)

    except Exception as e:
        st.error(f"Error fetching data: {e}")
        st.stop()

# ----------------------------------------------------------------
# Recommendation logic
# ----------------------------------------------------------------
st.write("---")
st.subheader("🌱 Recommendation")

recommendations = []
for param, (ideal_min, ideal_max) in CROP_CONDITIONS[crop].items():
    if "Soil" in param or "Altitude" in param:
        continue  # skip those since we don't have data
    local_avg = avg_data.get(param.replace("Annual ", "").replace("(mm)", "").strip(), None)
    if local_avg is not None and (ideal_min <= local_avg <= ideal_max):
        recommendations.append(True)
    else:
        recommendations.append(False)

suitability = "✅ YES — Suitable for growing!" if all(recommendations) else "❌ NO — Not ideal climate"
st.markdown(f"### {suitability}")

# ----------------------------------------------------------------
# Plot trends
# ----------------------------------------------------------------
st.write("---")
st.subheader("📊 Climate Trends at Your Location")

fig = px.line(df, x=df.index, y=df.columns, title="Monthly Climate Trends (2015–2024)")
st.plotly_chart(fig, use_container_width=True)

st.caption("Data Source: NASA POWER API | Visualization by Streamlit & Plotly")


2025-10-04 21:31:24.453 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:31:24.458 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:31:24.459 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:31:24.461 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:31:24.464 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:31:24.465 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:31:24.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:31:24.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

NameError: name 'avg_data' is not defined

In [18]:
import streamlit as st
import pandas as pd
import requests
import plotly.express as px

st.set_page_config(page_title="Crop Growth Suitability Demo", layout="wide")

# ----------------------------------------------------------------
# Crop database — easy to extend later
# ----------------------------------------------------------------
CROP_CONDITIONS = {
    "Rice": {
        "Temperature (°C)": (20, 35),
        "Annual Rainfall (mm)": (1000, 2000),
        "Soil pH": (5.5, 7.0),
        "Sunlight (MJ/m²/day)": (14, 20),
        "Relative Humidity (%)": (70, 80),
        "Altitude (m)": (0, 1500),
    },
    "Wheat": {
        "Temperature (°C)": (10, 25),
        "Annual Rainfall (mm)": (500, 1200),
        "Soil pH": (6.0, 7.5),
        "Sunlight (MJ/m²/day)": (10, 16),
        "Relative Humidity (%)": (50, 70),
        "Altitude (m)": (0, 3000),
    },
}

# ----------------------------------------------------------------
# NASA POWER API functions
# ----------------------------------------------------------------
def get_climate_data(lat, lon):
    """Fetch monthly climatology for multiple parameters from NASA POWER."""
    url = (
        f"https://power.larc.nasa.gov/api/temporal/monthly/point?"
        f"parameters=T2M,PRECTOTCORR,RH2M,ALLSKY_SFC_SW_DWN"
        f"&community=AG&longitude={lon}&latitude={lat}&format=JSON&start=2015&end=2024"
    )
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()["properties"]["parameter"]

    df = pd.DataFrame(data)
    df.index = pd.to_datetime(df.index)
    df.rename(
        columns={
            "T2M": "Temperature (°C)",
            "PRECTOTCORR": "Annual Rainfall (mm)",
            "RH2M": "Relative Humidity (%)",
            "ALLSKY_SFC_SW_DWN": "Sunlight (MJ/m²/day)",
        },
        inplace=True,
    )
    return df

# ----------------------------------------------------------------
# GUI Layout
# ----------------------------------------------------------------
st.title("🌾 Crop Growth Suitability Explorer")
st.write("Analyze if your location is suitable for growing different crops based on NASA POWER climate data.")

# Input controls
st.sidebar.header("User Input")
lat = st.sidebar.number_input("Latitude", value=22.0, format="%.2f")
lon = st.sidebar.number_input("Longitude", value=88.0, format="%.2f")
crop = st.sidebar.selectbox("Select Crop", list(CROP_CONDITIONS.keys()))

# ----------------------------------------------------------------
# Main content layout
# ----------------------------------------------------------------
col1, col2 = st.columns(2)

# Ideal conditions
with col1:
    st.subheader(f"✅ Ideal Conditions for {crop}")
    crop_df = pd.DataFrame(
        [(k, f"{v[0]}–{v[1]}") for k, v in CROP_CONDITIONS[crop].items()],
        columns=["Parameter", "Ideal Range"],
    )
    st.dataframe(crop_df, use_container_width=True)

# Fetch and show actual data
with col2:
    st.subheader("📍 Local Climate Data (NASA POWER)")
    df, avg_data, min_data, max_data = None, None, None, None
    try:
        df = get_climate_data(lat, lon)
        avg_data = df.mean().round(2)
        min_data = df.min().round(2)
        max_data = df.max().round(2)

        stats = pd.DataFrame(
            {"Min": min_data, "Avg": avg_data, "Max": max_data}
        ).reset_index()
        stats.rename(columns={"index": "Parameter"}, inplace=True)
        st.dataframe(stats, use_container_width=True)

    except Exception as e:
        st.error(f"⚠️ Error fetching data: {e}")

# ----------------------------------------------------------------
# Recommendation logic (only if data fetched)
# ----------------------------------------------------------------
if avg_data is not None:
    st.write("---")
    st.subheader("🌱 Recommendation")

    recommendations = []
    for param, (ideal_min, ideal_max) in CROP_CONDITIONS[crop].items():
        # Skip unmeasured parameters
        if param not in avg_data:
            continue
        local_avg = avg_data[param]
        if ideal_min <= local_avg <= ideal_max:
            recommendations.append(True)
        else:
            recommendations.append(False)

    suitability = (
        "✅ YES — Suitable for growing!"
        if all(recommendations)
        else "❌ NO — Not ideal climate"
    )
    st.markdown(f"### {suitability}")

    # ----------------------------------------------------------------
    # Plot trends
    # ----------------------------------------------------------------
    st.write("---")
    st.subheader("📊 Climate Trends at Your Location")
    fig = px.line(df, x=df.index, y=df.columns, title="Monthly Climate Trends (2015–2024)")
    st.plotly_chart(fig, use_container_width=True)

    st.caption("Data Source: NASA POWER API | Visualization by Streamlit & Plotly")
else:
    st.info("👆 Enter coordinates to fetch and visualize local climate data.")


2025-10-04 21:32:28.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:32:28.358 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:32:28.358 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:32:28.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:32:28.361 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:32:28.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:32:28.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-04 21:32:28.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar